### Get Vulnerabilities from OSV

In [1]:
import pandas as pd
import sys
sys.path.append('../utils')
import database as db

VUL_PATH = "vuls.json"
SFP_PATH = "spf.csv"

connect to CVEFIXes


In [2]:
import os
import glob
import json
# transform all vulnerabilities into a single json file
def data_transform(in_path, out_path):
    files = glob.iglob(in_path+"/*")
    result = list()
    for file in files:
        with open(file, 'r') as infile:
            result.append(json.load(infile))
    with open(out_path, 'w') as output_file:
        json.dump(result, output_file, indent=4)

if not os.path.exists(VUL_PATH):
    os.system("curl https://osv-vulnerabilities.storage.googleapis.com/crates.io/all.zip -o all.zip")
    os.system("unzip all.zip -d rust_vuls")
    data_transform("./rust_vuls", VUL_PATH)
    os.system("rm all.zip")
    os.system("rm -r rust_vuls")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1074k  100 1074k    0     0  1159k      0 --:--:-- --:--:-- --:--:-- 1178k


Archive:  all.zip
  inflating: rust_vuls/GHSA-2226-4v3c-cff8.json  
  inflating: rust_vuls/GHSA-2367-c296-3mp2.json  
  inflating: rust_vuls/GHSA-24g6-5rx7-58wj.json  
  inflating: rust_vuls/GHSA-28m8-9j7v-x499.json  
  inflating: rust_vuls/GHSA-28p5-7rg4-8v99.json  
  inflating: rust_vuls/GHSA-28ph-f7gx-fqj8.json  
  inflating: rust_vuls/GHSA-28r9-pq4c-wp3c.json  
  inflating: rust_vuls/GHSA-2969-8hh9-57jc.json  
  inflating: rust_vuls/GHSA-29hg-r7c7-54fr.json  
  inflating: rust_vuls/GHSA-29v7-3v4c-gf38.json  
  inflating: rust_vuls/GHSA-29xx-hcv2-c4cp.json  
  inflating: rust_vuls/GHSA-2f5j-3mhq-xv58.json  
  inflating: rust_vuls/GHSA-2gg5-7c4v-6xx2.json  
  inflating: rust_vuls/GHSA-2grh-hm3w-w7hv.json  
  inflating: rust_vuls/GHSA-2gxj-qrp2-53jv.json  
  inflating: rust_vuls/GHSA-2hvr-h6gw-qrxp.json  
  inflating: rust_vuls/GHSA-2jfv-g3fh-xq3v.json  
  inflating: rust_vuls/GHSA-2jq9-6xx7-3h29.json  
  inflating: rust_vuls/GHSA-2r6q-6c8c-g762.json  
  inflating: rust_vuls/GHSA-2rxc

### Get SFP mappings from cwe.mitre.org

In [3]:
if not os.path.exists(SFP_PATH):
    os.system("curl https://cwe.mitre.org/data/csv/888.csv.zip -o sfp.zip")
    os.system("unzip -q -p sfp.zip > spf.csv")
    os.system("rm sfp.zip")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  384k  100  384k    0     0   102k      0  0:00:03  0:00:03 --:--:--  102k


### Get  metadata of Rust packages from crates.io

In [26]:
if not os.path.exists("crates_io_db"):
    os.system("curl https://static.crates.io/db-dump.tar.gz -o crates_io_db.zip")
    os.system("unzip crates_io_db.zip -q -d crates_io_db")
    os.system("rm crates_io_db.zip")

df_crates = pd.read_csv("crates_io_db/data/crates.csv")
df_crates.to_sql(name="crates", con=db.conn, if_exists="replace", index=False)

df_categories = pd.read_csv("crates_io_db/data/categories.csv")
df_categories.to_sql(name="categories", con=db.conn, if_exists="replace", index=False)

df_crates_categories = pd.read_csv("crates_io_db/data/crates_categories.csv")
df_crates_categories.to_sql(name="crates_categories", con=db.conn, if_exists="replace", index=False)

df_versions = pd.read_csv("crates_io_db/data/versions.csv")
df_versions.to_sql(name="versions", con=db.conn, if_exists="replace", index=False)


### Format the vulnerabilities and merge vulnerability info

In [18]:
df_cve = pd.read_json(VUL_PATH)
df_sfp = pd.read_csv(SFP_PATH)
df_crates = pd.read_sql("SELECT name, repository FROM crates", con=db.conn)

In [19]:
def get_sfps_from_cwes():
    taxonomy = df_sfp["Affected Resources"].apply(lambda x: None if type(x)==float else (x.split("::TAXONOMY NAME:")))
    primary_cluster = {"SFP1":"Risky Values", "SFP2":"Unused Entities", "SFP3":"API", "SFP4":"Exception Management", "SFP5":"Exception Management", "SFP6":"Exception Management", \
                        "SFP7":"Memory Access", "SFP8":"Memory Access", "SFP9":"Memory Access", "SFP10":"Memory Access", "SFP11":"Memory Access", \
                        "SFP12":"Memory Management", "SFP13":"Resource Management", "SFP14":"Resource Management", "SFP15":"Resource Management", \
                        "SFP16":"Path Resolution", "SFP17":"Path Resolution", "SFP18":"Path Resolution", \
                        "SFP19":"Synchronization", "SFP20":"Synchronization", "SFP21":"Synchronization", "SFP22":"Synchronization",\
                        "SFP23":"Information Leak", "SFP24":"Tainted Input", "SFP25":"Tainted Input", "SFP26":"Tainted Input", "SFP27":"Tainted Input", \
                        "SFP28":"Entry Points", "SFP29":"Authentication", "SFP30":"Authentication", "SFP31":"Authentication", "SFP32":"Authentication", "SFP33":"Authentication", "SFP34":"Authentication",\
                        "SFP35":"Access Control", "SFP36":"Privilege", "SFP37":"Faulty Resource Release", "SFP38":"Failure to Release Memory"}
    def getSFP(x):
        if x is None:
            return x
        for t in x:
            if 'Software Fault Patterns' in t:
                sfp = t.split(':')[2]
                return primary_cluster[sfp] if sfp and 'SFP' in sfp else None
        return None
    taxonomy = taxonomy.apply(lambda x: getSFP(x) )
    # add expections
    taxonomy[908] = taxonomy[909] = "Exception Management"
    taxonomy[131] = taxonomy[787] = taxonomy[824]= taxonomy[119]= taxonomy[125] = "Memory Access"
    taxonomy[758] = "API"
    taxonomy[843] = "Risky Values"
    taxonomy[770] = taxonomy[772] = taxonomy[789] = "Resource Management"
    taxonomy[706] = "Path Resolution"
    taxonomy[362] = "Synchronization"
    taxonomy[668] = taxonomy[200] = taxonomy[203]= taxonomy[208]= taxonomy[377] = "Information Leak"
    taxonomy[129] = taxonomy[427] = taxonomy[172]= taxonomy[444]= taxonomy[198] = taxonomy[94] = taxonomy[351] = "Tainted Input"
    taxonomy[295] = "Authentication"
    taxonomy[279] = "Access Control"
    taxonomy[269] = "Privilege"
    taxonomy[327] = taxonomy[1240] = taxonomy[347] = "Cryptography"
    taxonomy[330] = taxonomy[338] = taxonomy[340] = "Predictability"
    taxonomy[657] = taxonomy[670] = taxonomy[682]= taxonomy[697]= taxonomy[188] = taxonomy[193] = taxonomy[835] = "Other"

    return taxonomy

In [20]:
from cvss import CVSS3
def get_vul_severity(data):
    temp = data.apply(lambda x: (x['affected'][0]['database_specific']['cvss'] if 'cvss' in x['affected'][0]['database_specific'] else None) if type(x['severity'])==float  else x['severity'][0]['score'] ,axis=1)
    return temp.apply(lambda x: CVSS3(x).severities()[0] if not x is None else None)
    
def get_vul_package(data):
    return pd.Series(list(d[0]['package']['name'] for d in data['affected']))

def get_vul_repo_url(data):
    return data['package'].apply(lambda x: df_crates['repository'][df_crates['name']==x].values[0] if len(df_crates['repository'][df_crates['name']==x].values)!=0 else None)

def get_vul_sfp_id(data):
    temp = data.apply(lambda x: x['affected'][0]['database_specific']['categories'] if type(x['database_specific'])==float else x['database_specific']['cwe_ids'], axis=1)
    taxonomy = get_sfps_from_cwes()
    res = ""
    for t in taxonomy[taxonomy=='Failure to Release Memory'].index:
        res = res + "CWE-"+str(t)+", "
    print(res)
    no_cat = 0
    cwe = 0
    category = 0

    for t in temp:
        if len(t)==0:
            no_cat= no_cat+1
        elif 'CWE' in t[0]:
            cwe = cwe+1
        else:
            category = category+1

    def get_sfp_from_cwe(x):
        cat2sfp = {"memory-exposure":"Memory Access", "memory-corruption":"Memory Management", "denial-of-service":"Resource Management", "file-disclosure":"Path Resolution",\
                    "thread-safety":"Synchronization", "format-injection":"Tainted Input", "privilege-escalation":"Privilege", "crypto-failure":"Cryptography", "code-execution":"Other"
                  }
        res = list()
        for cwe in x:
            if "CWE" in cwe:
                id = int(cwe.split('-')[1])
                if id in taxonomy.index and taxonomy[id]:
                    res.append(str(taxonomy[id])) 
            else:
                res.append(str(cat2sfp[cwe]))
        return res
    return temp.apply(lambda x: str(get_sfp_from_cwe(x)))

def get_vul_version(data):
    def get_version(affects):
        versions = list()
        for affect in affects:
            if "ranges" in affect:
                introduced = affect["ranges"][0]["events"][0]["introduced"] if "introduced" in affect["ranges"][0]["events"][0] else None
                fixed = affect["ranges"][0]["events"][1]["fixed"] if len(affect["ranges"][0]["events"])>1 and "fixed" in affect["ranges"][0]["events"][1] else None
                versions.append((introduced, fixed))
        return str(versions)

    return data["affected"].apply(lambda x:get_version(x) )

def get_vul_reference(data):
    references = data.pop('references')
    return pd.Series(str(list((ref['url'] for ref in d))) for d in references)


In [21]:
ordered_cve_columns = ['id', 'package', 'repo_url', 'sfp_id', 'modified', 'published', 'vul_version', 'summary', 'details', 'severity', 'references']
df_cve["severity"] = get_vul_severity(df_cve)
df_cve['package'] = get_vul_package(df_cve)
df_cve['repo_url'] = get_vul_repo_url(df_cve)
df_cve['sfp_id'] = get_vul_sfp_id(df_cve)
df_cve['vul_version'] = get_vul_version(df_cve)
df_cve['references'] = get_vul_reference(df_cve)
df_cve.drop(['aliases', 'database_specific', 'affected', 'schema_version', 'related'], axis=1, inplace=True)
df_cve = df_cve[ordered_cve_columns]
print(df_cve.head())
df_cve = df_cve.applymap(str)
df_cve.to_sql('cve_dup', db.conn, if_exists='replace', index=False)


                    id     package  \
0  GHSA-g4g4-3pqw-8m7f    rusqlite   
1  GHSA-5rrv-m36h-qwf8       chttp   
2    RUSTSEC-2021-0121     crypto2   
3    RUSTSEC-2022-0031       rulex   
4    RUSTSEC-2021-0064  cpuid-bool   

                                            repo_url                   sfp_id  \
0               https://github.com/rusqlite/rusqlite  ['Resource Management']   
1                  https://github.com/sagebind/chttp  ['Resource Management']   
2             https://github.com/shadowsocks/crypto2                       []   
3                     https://github.com/Aloso/rulex  ['Resource Management']   
4  https://github.com/RustCrypto/utils/tree/maste...                       []   

                   modified             published  \
0 2022-01-06 20:20:29+00:00  2022-01-06T22:02:01Z   
1 2021-08-19 21:22:50+00:00  2021-08-25T20:44:40Z   
2 2021-10-18 16:22:07+00:00  2021-10-08T12:00:00Z   
3 2022-06-26 20:01:26+00:00  2022-05-21T12:00:00Z   
4 2021-05-06 23:45

### Remove duplicate vulnerabilities

In [22]:
import pandas as pd
import ast

df_res = pd.DataFrame()
df_master = df_cve
# df_master = pd.read_csv("cve.csv")
packages = df_master.groupby('package').groups
print(len(df_master))
cnt = 0
for key, values in packages.items():
    # find duplicate vulnerability
    df_tmp = df_master.iloc[values]
    # get vulnerability related references
    ref = ast.literal_eval(df_tmp.iloc[0]['references'])
    ref = list(filter(lambda tmp: "nvd" in tmp or "rustsec" in tmp, ref))
    references = [ref]
    vuls = [[0]]
    for i in range(1, len(values)):
        flag = True
        tmp = ast.literal_eval(df_tmp.iloc[i]['references'])
        tmp = list(filter(lambda ref: "nvd" in ref or "rustsec" in ref, tmp))
        for idx, ref in enumerate(references):
            # same vulnerability or not
            if set(ref) & set(tmp):
                references[idx].extend(tmp)
                vuls[idx].append(i)
                flag = False
        if flag:
            vuls.append([i])
            references.append(tmp)

    # merge duplicate data and append it to new dataframe
    for vul_idx in vuls:
        # print(df_tmp.iloc[vul_idx])
        vul = {
            'id': [],
            'package': df_tmp.iloc[vul_idx[0]]['package'],
            'repo_url': df_tmp.iloc[vul_idx[0]]['repo_url'],
            'sfp_id': [],
            'modified': df_tmp.iloc[vul_idx[0]]['modified'],
            'published': df_tmp.iloc[vul_idx[0]]['published'],
            'vul_version': [],
            'summary': "",
            'details': "",
            'severity': "",
            'references': []
        }
        for vvul in vul_idx:
            vul['id'].append(df_tmp.iloc[vvul]['id'])
            vul['published'] = min(vul['published'], df_tmp.iloc[vvul]['published'])
            vul['sfp_id'].extend(ast.literal_eval(df_tmp.iloc[vvul]['sfp_id']))
            vul['references'].extend(ast.literal_eval(df_tmp.iloc[vvul]['references']))
            vul['vul_version'].extend(ast.literal_eval(df_tmp.iloc[vvul]['vul_version']))
            vul['sfp_id'] = list(set(vul['sfp_id']))
            vul['references'] = list(set(vul['references']))
            vul['vul_version'] = list(set(vul['vul_version']))

            vul['summary'] = df_tmp.iloc[vvul]['summary'] + '\n' + vul['summary']
            vul['details'] = df_tmp.iloc[vvul]['details'] + '\n' + vul['details']
            if df_tmp.iloc[vvul]['severity'] != 'nan':
                vul['severity'] = df_tmp.iloc[vvul]['severity']
        
        df_res = df_res.append(vul, ignore_index=True)
        
print(len(df_res))

974
508


### Dump vulnerabilities to database

In [23]:
df_res = df_res.applymap(str)
df_res.to_sql(name="cve2", con=db.conn, if_exists="replace", index=False)